In [238]:
from sklearn.metrics import auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

def specificity(y_ture,y_pred):
    MCM = confusion_matrix(y_ture, y_pred)
    tn_sum = MCM[0, 0]
    fp_sum = MCM[0, 1]

    tp_sum = MCM[1, 1]
    fn_sum = MCM[1, 0]

    Condition_negative = tn_sum + fp_sum

    Specificity = tn_sum / Condition_negative

    return Specificity

In [221]:
#用oversampling的方法进行建模
import numpy as np
import pandas
import pickle
from sklearn.linear_model import LogisticRegressionCV
import pickle as pl 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score,make_scorer
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
import pandas
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


file_afterlabel = pandas.read_excel("The Shanghai data with labels.xlsx")


file1_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==0]#select the female
file2_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==1]#select the male


file1_1_afterlabel_years = [file1_1_afterlabel.loc[round(file1_1_afterlabel["Age"]) == i] for i in range(10,19)]
file2_1_afterlabel_years = [file2_1_afterlabel.loc[round(file2_1_afterlabel["Age"]) == i] for i in range(10,19)]
#oversampling
for k in range(9):
    x_lin = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file1_1_afterlabel_years[k] = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file1_1_afterlabel_years[k] = pandas.concat([file1_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

for k in range(9):
    x_lin = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file2_1_afterlabel_years[k] = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file2_1_afterlabel_years[k] = pandas.concat([file2_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

#re-order for the nexe extraction
for k in range(9):
    file1_1_afterlabel_years[k].index = range(file1_1_afterlabel_years[k].shape[0])
    file2_1_afterlabel_years[k].index = range(file2_1_afterlabel_years[k].shape[0])
    
    
X_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:][file1_1_afterlabel_years[i].columns[1:23]] for i in range(9)]
X_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:][file2_1_afterlabel_years[i].columns[1:23]] for i in range(9)]



y_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]
y_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]

#construct the empty list
X_12_0_new_afterlabel_train = [[] for i in range(9)]  
X_12_0_new_afterlabel_test  = [[] for i in range(9)]
y_12_0_new_afterlabel_train = [[] for i in range(9)]
y_12_0_new_afterlabel_test = [[] for i in range(9)]

X_12_1_new_afterlabel_train = [[] for i in range(9)]  
X_12_1_new_afterlabel_test  = [[] for i in range(9)]
y_12_1_new_afterlabel_train = [[] for i in range(9)]
y_12_1_new_afterlabel_test = [[] for i in range(9)]

for i in range(9):
    X_12_0_new_afterlabel_train[i],X_12_0_new_afterlabel_test[i],y_12_0_new_afterlabel_train[i],y_12_0_new_afterlabel_test[i] = train_test_split(
    X_12_0_new_afterlabel_years[i],y_12_0_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_0_new_afterlabel_years[i])

#set random seed
for i in range(9):
    X_12_1_new_afterlabel_train[i],X_12_1_new_afterlabel_test[i],y_12_1_new_afterlabel_train[i],y_12_1_new_afterlabel_test[i] = train_test_split(
    X_12_1_new_afterlabel_years[i],y_12_1_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_1_new_afterlabel_years[i])


ABC_10_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_10_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_11_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_11_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_12_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_12_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_13_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_13_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_14_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_14_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_15_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_15_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_16_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_16_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_17_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_17_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)

ABC_18_0 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
ABC_18_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=5),
                         algorithm="SAMME.R",
                         n_estimators=10, learning_rate=1)
#cross validation for hyperparameters
param_dist = {
        'n_estimators':range(3,50,1),
        'learning_rate':np.linspace(0.01,2,20),
        }

grid_10_0 = GridSearchCV(ABC_10_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_10_1 = GridSearchCV(ABC_10_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_11_0 = GridSearchCV(ABC_11_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_11_1 = GridSearchCV(ABC_11_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_12_0 = GridSearchCV(ABC_12_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_12_1 = GridSearchCV(ABC_12_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_13_0 = GridSearchCV(ABC_13_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_13_1 = GridSearchCV(ABC_13_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_14_0 = GridSearchCV(ABC_14_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_14_1 = GridSearchCV(ABC_14_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_15_0 = GridSearchCV(ABC_15_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_15_1 = GridSearchCV(ABC_15_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_16_0 = GridSearchCV(ABC_16_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_16_1 = GridSearchCV(ABC_16_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_17_0 = GridSearchCV(ABC_17_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_17_1 = GridSearchCV(ABC_17_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_18_0 = GridSearchCV(ABC_18_0,param_dist,cv = 5, n_jobs = -1, verbose = 2)
grid_18_1 = GridSearchCV(ABC_18_1,param_dist,cv = 5, n_jobs = -1, verbose = 2)



#training the models
grid_10_0.fit(X_12_0_new_afterlabel_train[0],y_12_0_new_afterlabel_train[0].astype("int"))
grid_10_1.fit(X_12_1_new_afterlabel_train[0],y_12_1_new_afterlabel_train[0].astype("int"))

grid_11_0.fit(X_12_0_new_afterlabel_train[1],y_12_0_new_afterlabel_train[1].astype("int"))
grid_11_1.fit(X_12_1_new_afterlabel_train[1],y_12_1_new_afterlabel_train[1].astype("int"))

grid_12_0.fit(X_12_0_new_afterlabel_train[2],y_12_0_new_afterlabel_train[2].astype("int"))
grid_12_1.fit(X_12_1_new_afterlabel_train[2],y_12_1_new_afterlabel_train[2].astype("int"))

grid_13_0.fit(X_12_0_new_afterlabel_train[3],y_12_0_new_afterlabel_train[3].astype("int"))
grid_13_1.fit(X_12_1_new_afterlabel_train[3],y_12_1_new_afterlabel_train[3].astype("int"))

grid_14_0.fit(X_12_0_new_afterlabel_train[4],y_12_0_new_afterlabel_train[4].astype("int"))
grid_14_1.fit(X_12_1_new_afterlabel_train[4],y_12_1_new_afterlabel_train[4].astype("int"))

grid_15_0.fit(X_12_0_new_afterlabel_train[5],y_12_0_new_afterlabel_train[5].astype("int"))
grid_15_1.fit(X_12_1_new_afterlabel_train[5],y_12_1_new_afterlabel_train[5].astype("int"))

grid_16_0.fit(X_12_0_new_afterlabel_train[6],y_12_0_new_afterlabel_train[6].astype("int"))
grid_16_1.fit(X_12_1_new_afterlabel_train[6],y_12_1_new_afterlabel_train[6].astype("int"))

grid_17_0.fit(X_12_0_new_afterlabel_train[7],y_12_0_new_afterlabel_train[7].astype("int"))
grid_17_1.fit(X_12_1_new_afterlabel_train[7],y_12_1_new_afterlabel_train[7].astype("int"))

grid_18_0.fit(X_12_0_new_afterlabel_train[8],y_12_0_new_afterlabel_train[8].astype("int"))
grid_18_1.fit(X_12_1_new_afterlabel_train[8],y_12_1_new_afterlabel_train[8].astype("int"))



#save the trained models as pkl formats
with open('grid_10_0.pkl', 'wb') as f:
    pickle.dump(grid_10_0, f)
with open('grid_10_1.pkl', 'wb') as f:
    pickle.dump(grid_10_1, f)
    
with open('grid_11_0.pkl', 'wb') as f:
    pickle.dump(grid_11_0, f)
with open('grid_11_1.pkl', 'wb') as f:
    pickle.dump(grid_11_1, f)

with open('grid_12_0.pkl', 'wb') as f:
    pickle.dump(grid_12_0, f)
with open('grid_12_1.pkl', 'wb') as f:
    pickle.dump(grid_12_1, f)
    
with open('grid_13_0.pkl', 'wb') as f:
    pickle.dump(grid_13_0, f)
with open('grid_13_1.pkl', 'wb') as f:
    pickle.dump(grid_13_1, f)
    
with open('grid_14_0.pkl', 'wb') as f:
    pickle.dump(grid_14_0, f)
with open('grid_14_1.pkl', 'wb') as f:
    pickle.dump(grid_14_1, f)
    
with open('grid_15_0.pkl', 'wb') as f:
    pickle.dump(grid_15_0, f)
with open('grid_15_1.pkl', 'wb') as f:
    pickle.dump(grid_15_1, f)
    
with open('grid_16_0.pkl', 'wb') as f:
    pickle.dump(grid_16_0, f)
with open('grid_16_1.pkl', 'wb') as f:
    pickle.dump(grid_16_1, f)
    
with open('grid_17_0.pkl', 'wb') as f:
    pickle.dump(grid_17_0, f)
with open('grid_17_1.pkl', 'wb') as f:
    pickle.dump(grid_17_1, f)
    
with open('grid_18_0.pkl', 'wb') as f:
    pickle.dump(grid_18_0, f)
with open('grid_18_1.pkl', 'wb') as f:
    pickle.dump(grid_18_1, f)


Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidates, totalling 4700 fits
Fitting 5 folds for each of 940 candidat

In [264]:
import numpy as np
import pandas
import pickle
import random
from sklearn.linear_model import LogisticRegressionCV
import pickle as pl 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#evaluate the performance of the series of talent prediction models

file_afterlabel = pandas.read_excel("The Shanghai data with labels.xlsx")


file1_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==0]#select the female
file2_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==1]#select the male

#进行oversampling
for k in range(9):
    x_lin = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file1_1_afterlabel_years[k] = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file1_1_afterlabel_years[k] = pandas.concat([file1_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

for k in range(9):
    x_lin = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file2_1_afterlabel_years[k] = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file2_1_afterlabel_years[k] = pandas.concat([file2_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

#re-order
for k in range(9):
    file1_1_afterlabel_years[k].index = range(file1_1_afterlabel_years[k].shape[0])
    file2_1_afterlabel_years[k].index = range(file2_1_afterlabel_years[k].shape[0])

index_name = file1_1_afterlabel.columns[1:23]

file1_1_afterlabel_years = [file1_1_afterlabel.loc[round(file1_1_afterlabel["Age"]) == i] for i in range(10,19)]#分别选出来10-18岁
file2_1_afterlabel_years = [file2_1_afterlabel.loc[round(file2_1_afterlabel["Age"]) == i] for i in range(10,19)]

X_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:][file1_1_afterlabel_years[i].columns[1:23]] for i in range(9)]
X_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:][file2_1_afterlabel_years[i].columns[1:23]] for i in range(9)]



y_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]
y_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]

X_12_0_new_afterlabel_train = [[] for i in range(9)]  
X_12_0_new_afterlabel_test  = [[] for i in range(9)]
y_12_0_new_afterlabel_train = [[] for i in range(9)]
y_12_0_new_afterlabel_test = [[] for i in range(9)]

X_12_1_new_afterlabel_train = [[] for i in range(9)]  
X_12_1_new_afterlabel_test  = [[] for i in range(9)]
y_12_1_new_afterlabel_train = [[] for i in range(9)]
y_12_1_new_afterlabel_test = [[] for i in range(9)]

for i in range(9):
    X_12_0_new_afterlabel_train[i],X_12_0_new_afterlabel_test[i],y_12_0_new_afterlabel_train[i],y_12_0_new_afterlabel_test[i] = train_test_split(
    X_12_0_new_afterlabel_years[i],y_12_0_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_0_new_afterlabel_years[i])

#set random seed
for i in range(9):
    X_12_1_new_afterlabel_train[i],X_12_1_new_afterlabel_test[i],y_12_1_new_afterlabel_train[i],y_12_1_new_afterlabel_test[i] = train_test_split(
    X_12_1_new_afterlabel_years[i],y_12_1_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_1_new_afterlabel_years[i])

y = [[] for i in range(9)]
y_1 = [[] for i in range(9)]

proba_test = [[] for i in range(9)]
proba_test_1 = [[] for i in range(9)]

y_pred = [[] for i in range(9)]
y_pred_1 = [[] for i in range(9)]
for i in range(9):
    if i == 0:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_10_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_10_0.predict_proba(X_12_0_new_afterlabel_test[i])
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_10_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_10_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 1:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_11_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_11_0.predict_proba(X_12_0_new_afterlabel_test[i])        
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_11_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_11_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 2:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_12_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_12_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_12_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_12_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 3:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_13_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_13_0.predict_proba(X_12_0_new_afterlabel_test[i])            
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_13_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_13_1.predict_proba(X_12_1_new_afterlabel_test[i])        
    if i == 4:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_14_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_14_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_14_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_14_1.predict_proba(X_12_1_new_afterlabel_test[i])            
    if i == 5:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_15_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_15_0.predict_proba(X_12_0_new_afterlabel_test[i])                 
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_15_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_15_1.predict_proba(X_12_1_new_afterlabel_test[i])        
    if i == 6:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_16_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_16_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_16_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_16_1.predict_proba(X_12_1_new_afterlabel_test[i])       
    if i == 7:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_17_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_17_0.predict_proba(X_12_0_new_afterlabel_test[i])                      
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_17_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_17_1.predict_proba(X_12_1_new_afterlabel_test[i])     
    if i == 8:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = grid_18_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = grid_18_0.predict_proba(X_12_0_new_afterlabel_test[i])           
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = grid_18_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = grid_18_1.predict_proba(X_12_1_new_afterlabel_test[i])     
        
for i in range(9):
    roc_sum = [roc_curve(y[i],proba_test[i][:,1]) for i in range(9)]
    fpr_test = [roc_sum[i][0] for i in range(9)]
    tpr_test = [roc_sum[i][1] for i in range(9)]
    threshold = [roc_sum[i][2] for i in range(9)]
    roc_auc_test = [auc(fpr_test[i],tpr_test[i]) for i in range(9)]#calculate the AUC area
    
    roc_sum_1 = [roc_curve(y_1[i],proba_test_1[i][:,1]) for i in range(9)]
    fpr_test_1 = [roc_sum_1[i][0] for i in range(9)]
    tpr_test_1 = [roc_sum_1[i][1] for i in range(9)]
    threshold_1 = [roc_sum_1[i][2] for i in range(9)]
    roc_auc_test_1 = [auc(fpr_test_1[i],tpr_test_1[i]) for i in range(9)] 
        
data_0 = pandas.DataFrame({"Recall":[recall_score(y[i], y_pred[i]) for i in range(9)],"Precision":[precision_score(y[i], y_pred[i]) for i in range(9)],
                         "Accuracy":[accuracy_score(y[i], y_pred[i]) for i in range(9)],"Specificity":[specificity(y[i], y_pred[i]) for i in range(9)],
                        "AUC":roc_auc_test,"F1 score":[f1_score(y[i], y_pred[i]) for i in range(9)]})
data_1 = pandas.DataFrame({"Recall":[recall_score(y_1[i], y_pred_1[i]) for i in range(9)],"Precision":[precision_score(y_1[i], y_pred_1[i]) for i in range(9)],
                         "Accuracy":[accuracy_score(y_1[i], y_pred_1[i]) for i in range(9)],"Specificity":[specificity(y_1[i], y_pred_1[i]) for i in range(9)],
                         "AUC":roc_auc_test_1,"F1 score":[f1_score(y_1[i], y_pred_1[i]) for i in range(9)]})

        召回率       精准率       准确率       特异性     AUC面积       f1值
0  1.000000  0.538462  0.828571  0.785714  1.000000  0.700000
1  1.000000  0.705882  0.919355  0.900000  0.980000  0.827586
2  1.000000  0.640000  0.878378  0.844828  0.956897  0.780488
3  1.000000  0.588235  0.898551  0.881356  0.977966  0.740741
4  1.000000  0.666667  0.921569  0.906977  0.982558  0.800000
5  1.000000  0.777778  0.950000  0.939394  0.995671  0.875000
6  1.000000  0.454545  0.785714  0.739130  1.000000  0.625000
7  0.666667  0.571429  0.800000  0.842105  0.947368  0.615385
8  1.000000  0.571429  0.785714  0.700000  1.000000  0.727273
   召回率       精准率       准确率       特异性     AUC面积       f1值
0  1.0  0.714286  0.953488  0.947368  1.000000  0.833333
1  1.0  0.600000  0.893333  0.873016  0.932540  0.750000
2  1.0  0.523810  0.887640  0.871795  1.000000  0.687500
3  1.0  0.666667  0.940594  0.932584  0.984082  0.800000
4  1.0  0.545455  0.896907  0.882353  1.000000  0.705882
5  1.0  0.526316  0.886076  0.869565  